In [1]:
import pandas as pd
import numpy as np
import sqlite3
from tqdm.notebook import tqdm

In [ ]:
#db = sqlite3.connect('data/NPPES_Data_Dissemination.sqlite')
query = "SELECT * FROM hop"

with sqlite3.connect('data/NPPES_Data_Dissemination.sqlite') as db: 
    hop_sqlite = pd.read_sql(query, db)
    
hop_sqlite

In [ ]:
#db = sqlite3.connect('data/NPPES_Data_Dissemination.sqlite')
query = "SELECT * FROM nppes"

with sqlite3.connect('data/NPPES_Data_Dissemination.sqlite') as db: 
    nppes_sqlite = pd.read_sql(query, db)
    
nppes_sqlite

First, build a profile of providers referring patients to the major hospitals in Nashville. Are certain specialties more likely to refer to a particular hospital over the others?

In [ ]:
db.close

In [ ]:
db = sqlite3.connect('data/NPPES_Data_Dissemination.sqlite')

providers_query = """

SELECT NPI, [ProviderOrganizationName(LegalBusinessName)], 
ProviderBusinessPracticeLocationAddressPostalCode, HealthcareProviderTaxonomyCode,
HealthcareProviderPrimaryTaxonomySwitch 

FROM [nppes] WHERE EntityTypeCode = 2 AND ProviderBusinessPracticeLocationAddressCityName = 'NASHVILLE' 
AND ProviderBusinessPracticeLocationAddressStateName = 'TN' AND HealthcareProviderTaxonomyCode IS NOT NULL
AND HealthcareProviderPrimaryTaxonomySwitch = 'Y'
"""
    
provider = pd.read_sql(providers_query, db)
    
provider

In [ ]:
db.close

In [ ]:
taxonomy_nucc = pd.read_csv('data/nucc_taxonomy_230.csv', encoding= 'unicode_escape')

##taxonomy_nucc1 = taxonomy_nucc[taxonomy_nucc['Grouping'] == 'Hospital Units']
taxonomy_nucc

In [ ]:
provider = pd.DataFrame(provider_sqlite)
provider = provider.merge(taxonomy_nucc, left_on = "HealthcareProviderTaxonomyCode", right_on = "Code")
provider_hospitals = provider[provider['Grouping'] == 'Hospitals'] 
provider_hospitals

Merging data to be able to get 3. question done. 

In [ ]:
provider.to_sql('provider', db, if_exists = 'replace', index = False)

- Determine which professionals Vanderbilt Hospital should reach out to in the Nashville area to expand their own patient volume.
First, research which professionals are sending significant numbers of patients only to competitor hospitals (such as TriStar Centennial Medical Center).

In [2]:
db = sqlite3.connect('data/NPPES_Data_Dissemination.sqlite')

competitors = """
SELECT h.from_npi, h.to_npi, h.patient_count, h.transaction_count, ProviderBusinessPracticeLocationAddressCityName AS city_name, [ProviderOrganizationName(LegalBusinessName)] AS Business_name, 
ProviderFirstName AS First_Name, [ProviderLastName(LegalName)] AS Last_Name 
        
FROM nppes AS n
INNER JOIN hop AS h
ON n.NPI = h.to_npi

"""
with sqlite3.connect('data/NPPES_Data_Dissemination.sqlite') as db: 
    competitors = pd.read_sql(competitors, db)
    

In [3]:
competitors

,from_npi,to_npi,patient_count,transaction_count,city_name,Business_name,First_Name,Last_Name
0,1295726032,1922001957,83,89,NASHVILLE,None,RICHARD,PRESLEY
1,1578584199,1922001957,129,149,NASHVILLE,None,RICHARD,PRESLEY
2,1295726032,1922001957,83,89,NASHVILLE,None,RICHARD,PRESLEY
3,1578584199,1922001957,129,149,NASHVILLE,None,RICHARD,PRESLEY
4,1295726032,1922001957,83,89,NASHVILLE,None,RICHARD,PRESLEY
...,...,...,...,...,...,...,...,...
53854135,1811955917,1104850130,52,90,NASHVILLE,HEIKKI E KOSTAMAA MD PC,None,None
53854136,1861479545,1104850130,195,287,NASHVILLE,HEIKKI E KOSTAMAA MD PC,None,None
53854137,1871529982,1104850130,80,81,NASHVILLE,HEIKKI E KOSTAMAA MD PC,None,None
53854138,1922039346,1104850130,912,2042,NASHVILLE,HEIKKI E KOSTAMAA MD PC,None,None
